### 一. 依赖状态的类实现
#### 1. 什么叫依赖状态的类
* 如果类中的一些操作有基于状态的前提. 则这个类是依赖状态的
* 类库中包含许多依赖状态的同步类, 比如FutureTask, Semaphore, BlockingQueue等.  
  例如不能从一个空队列中删除元素, 或者不能获取一个尚未结束的任务的计算结果
  
#### 2. 多线程下状态依赖 - 等待条件变为真
* 单线程程序中, 条件分支依赖于某个状态, 如果这个条件不满足, 可以立刻失败, 因为该状态不会发生改变  
* 多线程程序中, 基于状态的条件可能会被其他线程修改, 所以不能立刻进入失败分支, 而是可以**等待条件变为真**

#### 3. 如何进行高效的条件等待
&nbsp;&nbsp;&nbsp;&nbsp;为了突出高效的条件等待方法 ,先来介绍一些低效的条件等待方法. 如下方法效率依次增高, 直到最后一种方法效率为最高:  
一个统一的场景, `有界队列实现`; 有界队列的put和take往往包含一个前提条件,不能从空队列中获取元素, 也不能将元素放入已满的队列之中. 当条件不满足时, 可以阻塞直到对象进入正确的状态, 或抛出异常或返回一个错误状态. 如下是所有有界队列实现的父类, 定义了公用的`isFull()`和`isEmpty()`
<img src="img/aqs1.png" width="85%">
1. 方法一 : **将条件检测的失败状态传递给调用者(抛异常或返回一个错误值)**   
   该方法下, 如果条件依赖检查失败, 直接在方法中抛异常通知调用者. 这看起来实现简单, 但使用起来有2个问题
    1. 依赖不满足只是当下不满足, 而不是发生了异常
    2. 调用者每次调用必须做好捕获异常的准备, 且在异常发生时主动重试<img src="img/aqs11.png" width="60%">
    3. 客户调用者捕获异常后, 可以有两种重试选择 :   
        * 自行进入休眠   
        catch块中进行`Thread.sleep()`
        ```java
        while(true){
            try{
               V item = buffer.get();
               break;  // 未发生异常跳出循环
            }catch (Exception e){
               Thread.sleep(SLEEP_GRANULARITY)  // 自行休眠
            }
        }
        ```
        * 自旋等待(忙等待)  
        catch中调用`Thread.yield()`, 当调度器给该线程一个时间片时, yield选择让出处理器而不是消耗完整个CPU时间片
        ```java
        while(true){
            try{
               V item = buffer.get();
               break;  // 未发生异常跳出循环
            }catch (Exception e){
               Thread.yield();  // 自旋等待
            }
        }
        ```
* 方法二: **通过轮训和休眠的方式, 勉强的解决状态依赖问题**  
    1. 该方法也是先进行条件检查, 然后执行方法; 但会在检查失败后自行重试      
       首先, 状态条件的检查必须在锁的保护下进行.     
       其次, 如果测试失败, 当前执行的线程应该释放锁并休眠一段时间, 让其他线程能够访问内部的buf; 当线程醒来后, 将重新请求锁并再次进行重试. 因而, 线程将反复的在条件检查,休眠,获取锁释放锁的等待过程中切换.     
    2. 该方法实际将跑出异常那个方法中, 调用者的重试逻辑放到了方法内部进行; 使方法内部处理重试逻辑, 客户端只负责简单调用.<img src="img/aqs12.png" width="68%">
    3. 睡眠时间间隔选择   
       要选择合适的睡眠时间, 需要在响应性和CPU使用率之间权衡. 睡眠时间小, 响应性就高, 但消耗的CPU资源也高   
    4. 展望  
       这种通过轮训和睡眠实现阻塞的过程需要进行巨大努力. 如果存在某种挂起线程的方法, 能够保证当依赖条件为真时才立即醒来, 则将极大地提高效率, 这正是条件队列实现的功能.  
       
* 方法三: **使用条件队列**
    1. 条件队列:    
       条件队列的名字来源于: 使"等待某个条件为真"的线程加入到一个队列中.   
       一般队列中每个元素都是数据, 而**条件队列中每个元素是一个线程**
    2. 一个对象就是一个条件队列   
        * 正如每个对象都可以是一个锁一样, 每个对象也可以作为一个条件队列; 对象的wait(), notify()构成了使用内部条件队列的API   
        * 对象的内置锁和内置条件队列是相互关联的, 要调用对象X内置条件队列的任意一个方法, 必须先持有对象X的内置锁. 这也是因为 "等待由状态构成的条件" 和 "维护状态的一致性" 两种机制必须紧密地绑定在一起
        * wait():   
            1. 会释放锁, 并请求操作系统挂起当前线程. 使其他线程能够获取这个锁并修改对象的状态.   
            2. wait()醒来时, 将在返回之前重新获取锁.   
            3. wait()通常写在while循环内部. 由于代码可能出现bug, 线程不一定是在条件满足后醒来, 还可能在条件处于任意状态下被错误唤醒, 因此线程醒来后需要重新判断依赖条件是否为true  
        * 使用条件队列完成条件状态依赖, 和轮训休眠拥有相同的语义. 如果某个功能使用 "轮训和休眠" 无法完成, 那么使用条件队列也无法完成;<img src="img/aqs13.png" width="70%">

### 二. 使用条件队列的注意事项  
条件队列使得构建高响应性对的状态依赖类变得容易, 但很容易造成误用; 因此使用时必须注意以下几点  
#### 2.1 条件谓词
1. 什么是条件谓词   
    * 条件谓词: 是使某个操作变为状态依赖操作的前提. 要想正确使用条件队列, 关键是找出对象在哪个条件谓词上等待  
    * 有界队列中, 只有当缓存不为空的时候, take()才能执行, 否则必须等待; 因此, take()的条件谓词是"队列不为空"  
      同样, put()只有在队列不满的情况下才能执行, 否则必须等待, 因此, put()的条件谓词是"队列不满"   
      
2. 怎么写条件谓词  
   条件谓词是由类中各个状态变量构成的表达式  
   
3. 条件谓词如何实现条件等待  
    * 条件等待中有一个重要的三元组关系: 加锁; wait()方法; 条件谓词
        * 锁加在状态变量上
        * 状态变量表达式构成了条件谓词
        * 一个条件谓词若判断为false, 则当前线程加入锁对应的条件队列并挂起
    * 因为条件谓词中包含多个状态变量, 而这些状态变量需要由同一个锁来保护, 所以在判断条件谓词为true或false之前必须先持有这个锁   
    * 基于以上2点, `状态变量->条件谓词->条件等待队列->锁`之间的关系克表示如下<img src="img/aqs11122.png" width="90%">  
    
#### 2.2 过早唤醒
1. 什么情况会引起过早唤醒(条件谓词不满足时就被唤醒)     
由于条件队列可以和多个条件谓词配合使用, 因此, 当一个挂起在wait()上的线程被notifyALL()唤醒时, 并不意味着该线程正在等待的条件谓词变成了true 
 而且wait()方法还能执行有限时间的挂起, 超市后自动就会唤醒. 因此, 将条件谓词的检查放在while循环中是必要的的

#### 2.3 丢失的信号
1. wait()和notify()要有先后顺序?  
 如果线程A在一个条件队列上执行了notify(), 而线程B随后才在这个条件队列上等待, 那么线程B将不会被线程A唤醒, 只能等待下一个notify()信号到来. 这就是线程A的信号丢失

#### 2.4 通知
1. **条件等待的必要操作: 通知**  
    1. 前面介绍的都是条件谓词的检查, 条件谓词不满足时线程加入条件队列进行等待; 还有另一半重要内容: 通知: 
        * notify() : 从条件列中选择一个线程唤醒
        * notifyAll() : 把条件队列中所有的线程全部唤醒  
    2. 发出通知的线程应该尽快释放锁  
       线程调用notify()后, 虽然唤醒了其它线程进入锁的争抢, 但并未释放锁; 所以应该尽快释放锁, 让其它线程能够立马争取锁, 争取成功的线程会从wait()处的代码唤醒  
    2. 上面的有界队列中, 如果队列为空, 则take()操作陷入等待, 当队列不为空时, take()操作返回; 因此必须确保每条会导致队列非空的操作都能发出一个通知, 通知take()可以返回了. 由于导致队列非空的操作时put(), 所以每次put()都要发起对条件队列的通知notifyAll()   
    
2. **使用notify()通知时危险的**   
  由于线程会因为不同的条件谓词不满足而进入同一个条件队列, 如果使用notify()通知而不是notifyAll()通知就会很危险, 因为单一的通知很容易导致类似于信号丢失的问题: 
    ``` 
    比如, 线程A在条件队列中等待条件谓词PA, 线程B在条件队列中等待条件谓词PB, 此时线程C执行一个notify(), JVM选择线程A唤醒, 但线程A发现条件谓词并不满足进而进入挂起等待, 而其实线程C的操作可以让条件谓词PB满足, 但是线程B没能被唤醒, 造成了线程C的通知丢失
    ```
3. 单次通知(notify())的严格条件    
通过以上分析, 只有下面2个条件同时满足时, 才可以使用notify()进行通知: 
    * **只有1个条件谓词和条件队列相关**,
    * **且条件变量上的每次通知最多只能唤醒一个线程**
    
 次时才能使用notify()发出通知

3. **条件通知 - 使用条件优化notifyAll()通知**   
有些开发人员认为"完全使用notifyAll()代替notify()"是低效的, 大多数情况下这种低效影响很小, 但有时候可能很大. 比如: 
```
假设有10个线程在条件队列上等待, 当某个线程执行notifyAll()后, 10个线程全部唤醒, 并在锁上进行竞争. 然而, 他们中的大多数唤醒后仍发现条件谓词不满足而又重回到挂起, 这会产生大量的上下文切换和锁竞争
```
 事实上, 条件等待版本的有界队列实现中, put()和take()的通知机制是保守的: 每当放入一个对象或从队列中取走一个对象就发起一次通知. 我们可以对其优化:
     * 首先, 仅当队列从空变为非空时, 或者从满变为不满时才释放一个线程
     * 并且, 仅当put()或take()影响到这些状态转换时才发出通知<img src="img/aqs114.png" width="70%">
 条件通知(条件判断下的notifyAll())和单次通知(notify)可以提升性能但却很难实现, 因此使用时应当谨慎, 只有在保证程序在notifyAll()下已经正常执行后, 再使用这个优化措施

### 三. 显式的Condition对象
Condition是一种广义的锁, 当然也是一种广义的条件队列; 有时使用内置锁过于灵活也可以使用广义锁  

#### 3.1 一个广义锁可以关联多个条件队列  
1. 内置锁的弊端和内置锁notifyAll()的局限   
内置条件队列的一个缺陷时, 每个内置锁只能和一个内置条件队列关联. 像上面的有界队列, 多个线程可能会在同一个条件队列上等待不同的条件谓词, 这使得无法满足"在调用notifyAll()时唤醒的的线程都是同一类型的需求". 如果要编写带有多个条件谓词的并发对象, 就可以使用显式的Lock和Condition, 而不是内置锁和条件队列

2. ReentrantLock与Condition对象  
    * 一个Condition和一个Lock关联在一起, 但是一个Lock可以和多个Condition关联在一起
    * 每个Condition代表一个条件队列, Conditiopn对象会继承Lock对象的公平性: 对于公平锁, 线程会依照FIFO的顺序从Condition.await()中被释放
    ```
    Condition对象的等待和通知方法分别是: await(),signal()和signalAll(). 但是因为Condition扩展了Object, 所以它也有wait()和notify()方法. 使用时一定要调用正确的方法 - await()和signal()
    ```
    
3. Condition实现的有界队列  
    * 因为有界队列有2个谓词, "非空"和"非满", 所以可以用2个条件谓词(2个Condition对象): notFull和notEmpty;   
      当队列为空时, take()挂起并等待notEmpty, 此时put()向notEmpty发送信号, 解除阻塞在take()中的线程   <img src="img/aqs21.png" width="83%">
    * 使用多个Condition时, 比分析一个使用单一内部队列加多个条件谓词的类简单的多. 通过把条件谓词分开并放入到2个等待线程集合中, Condition更容易满足单次通知的需求. signal()比signalAll()更高效, 它能极大的减少每次唤醒操作产生的上写文切换和锁请求次数
    
#### 3.2 RentrantLock和Semaphore的共性

### 二. 如何使用AbstractQueuedSynchronizer构建同步类
<img src="img/aqsdemo.png" width="65%">
1. `boolean tryAcquire(int arg)`
2. `boolean tryRelease(int arg)`


1. `int tryAcquireShared(int arg)`
2. `boolean tryReleaseShared(int arg)`

### 三. 几个问题: 
#### 1. FutureTask如何实现阻塞版的get(), 等待任务完毕才能返回
1. java1.8中实现的FutureTask使用LockSupport实现任务的等待??  
    * (1) LockSupport中定义了线程阻塞,唤醒的方法; 定义了获取线程阻塞在哪个object上等方法
    * (2) `Unsafe`类的作用:
        1. 阻塞或唤醒线程 : 
            * `public native  void park(boolean isAbsolute,long time)`
            * `public native  void unpark(Object thread)`
        2. 获得内存地址上的值 : 
            * `public native double getDoubleVolatile(Object o,long offset)`
            * `public native float getFloat(long address)`
            * `public native float getFloat(Object o,long offset)`
        3. 修改内存地址的值
            * `public native  void putDouble(long address,double x)`
            * `public native  void putDouble(Object o,long offset,double x)`

     上面几个方法中的offset值内存的偏移地址, 此外Unsafe中定义了不同数据类型的单个偏移地址. 比如
     ```java
     ARRAY_LONG_INDEX_SCALE = theUnsafe.arrayIndexScale(long[].class);
     ```
     
2. FutureTask阻塞版get()的实现        
    1. FutureTask对任务的状态做了定义, 每个状态都是一个volatile的int值.    
       <img src="img/futuretask1.png" width="40%">
    2. 再来看一下FutureTask内部的一些属性:
        ```java
        /** 定义任务 */
        private Callable<V> callable;
        /** 任务执行的结果, 供get()发挥 */
        private Object outcome; // non-volatile, protected by state reads/writes
        /** 执行任务的线程 */
        private volatile Thread runner;
        /** 等待获取get的线程栈 */
        private volatile WaitNode waiters;
        ```
    2. FutureTask的run方法 :   
       在线程池中的本地线程执行Callable的call方法
       ```java
        public void run() {
            if (state != NEW ||
                // 1. CAS设置该FutureTask的runner对象为当前线程 runnerOffset=UNSAFE.objectFieldOffset(k.getDeclaredField("runner")
                !UNSAFE.compareAndSwapObject(this, runnerOffset, null, Thread.currentThread()))
                return;
            try {
                Callable<V> c = callable;
                if (c != null && state == NEW) {
                    V result;
                    boolean ran;
                    try {
                        result = c.call();  // 2. 执行任务
                        ran = true;
                    } catch (Throwable ex) {
                        result = null;
                        ran = false;
                        setException(ex);
                    }
                    if (ran)
                        set(result);
                }
            } finally {
                // runner must be non-null until state is settled to prevent concurrent calls to run()
                runner = null;
                // state must be re-read after nulling runner to prevent leaked interrupts
                int s = state;
                if (s >= INTERRUPTING)
                    handlePossibleCancellationInterrupt(s);
            }
        }
       ```
    2. get()方法首先判断当前任务的任务的状态:  
        * 如果还未运行完(<=Compiled), 则自旋等待任务运行完(awaitDone): 否则返回运行结果
            1. 在FutureTask结果的等待栈中加入一个新的节点WaitNode
            ```java
            static final class WaitNode {
                volatile Thread thread;
                volatile WaitNode next;
                WaitNode() { thread = Thread.currentThread(); }
            }
            ```
            2. 如果任务状态是COMPLETING, 表示还未运行完, 进入自旋等待(while(true){Thread.yield})
            ```java
            private int awaitDone(boolean timed, long nanos) throws InterruptedException {
                final long deadline = timed ? System.nanoTime() + nanos : 0L;  // 如果是有闲时间等待的话计算deadline
                WaitNode q = null;
                boolean queued = false;
                for (;;) {
                    if (Thread.interrupted()) {
                        removeWaiter(q);
                        throw new InterruptedException();
                    }

                    int s = state;
                    if (s > COMPLETING) {
                        if (q != null)
                            q.thread = null;
                        return s;
                    }
                    else if (s == COMPLETING) // 还未运行完, 自旋等待
                        Thread.yield();
                    else if (q == null)
                        q = new WaitNode();
                    else if (!queued)
                        //像等待栈中加入一个节点. waitersOffset = UNSAFE.objectFieldOffset(k.getDeclaredField("waiters"));
                        queued = UNSAFE.compareAndSwapObject(this, waitersOffset, q.next = waiters, q);
                    else if (timed) {
                        nanos = deadline - System.nanoTime();
                        if (nanos <= 0L) {
                            removeWaiter(q);
                            return state;
                        }
                        LockSupport.parkNanos(this, nanos);
                    }
                    else
                        LockSupport.park(this);
                    }
                }
            ```
    3.  cancel取消任务:
        因为runner属性记录了当前运行任务的线程, 所以调用运行任务线程的interupt()方法.所以, 只有在Callable的call()方法中设计了响应中断, 才能让cancel()方法生效. 
        ```java
        while(Thread.currentThread().isInterrupted() && ! flag)
        ```